Liebe Kollegen,

im Zuge der Entwicklung eines Tools zur Annotation radiologischer Bilder möchten wir einen kleinen Feldversuch durchführen. Dazu brauchen wir Eure Hilfe.

Euch werden 50 konventionelle Thorax-Röntgen angezeigt. Zu jedem Bild müsst ihr lediglich entscheiden, ob die gefragte Pathologie vorliegt oder nicht. Es gibt nur die Antwortmöglichkeiten „Ja“ oder „Nein“. Auf einen „Ich weiss nicht“-Button wurde absichtlich verzichtet. 

Entscheidet euch also für die Antwort, die Euch unter der gegebenen einfachen Funktionalität des Viewers (einfacher Zoom, kein Fenstern) am wahrscheinlichsten erscheint.  Wir möchte euch bitten, die Aufgabe in einer Sitzung durchzuführen (Dauer: ca. 8 Minuten).

Um auf einen Bereich zu zoomen kann man mit der Maus auf das Bild klicken und eine Region aufziehen. Um wieder heraus zu zoomen: Doppelklick auf das Bild.

Am Ende wird euch über ein Freitextfeld noch die Möglichkeit zu Kommentaren gegeben. Über den zweiten Link („Result“) könnt ihr Eure Antworten mit der Ground Truth aus dem öffentlichen NIH-Datensatz vergleichen.

In [ ]:
%%HTML
<!-- Global site tag(gtag.js) - Google Analytics - ->
<script async src = "https://www.googletagmanager.com/gtag/js?id=UA-123889833-1" > </script >
<script >
    window.dataLayer = window.dataLayer | | []
    function gtag(){dataLayer.push(arguments)
                    }
    gtag('js', new Date())

    gtag('config', 'UA-123889833-1')
</script >

In [ ]:
import os, sys
from time import time
sys.path.append('.')
import jupyanno as ja
from jupyanno.widgets import BinaryClassTask, MultiClassTask
from jupyanno.sheets import submit_response, get_questions
import ipywidgets as ipw
ja.setup_appmode()

In [ ]:
from IPython.display import display
USERNAME = ja._get_user_id()
HOSTNAME = os.environ.get('HOSTNAME', 'anon')
BINARY_TASK = True
UNKNOWN_OPTION = None
ITEMS_TO_PROCESS = 50
SHOW_WINDOW_TOOLS = False

In [ ]:
annotation_task = ja.read_task_file('task.json')

In [ ]:
category_ids = sorted(
    annotation_task.data_df[annotation_task.label_col].unique().tolist())
if (UNKNOWN_OPTION is not None) and not BINARY_TASK:
    category_ids += [UNKNOWN_OPTION]

In [ ]:
START_TASK_TIME = time()
FORM_URL = annotation_task.task['google_forms']['form_url']
anno_questions = get_questions(FORM_URL)
COMMIT_ID = !git rev-parse HEAD

def submit_label(task_result):
    """
    the connection to the backend
    """
    submit_response(FORM_URL, anno_questions,
                    annotator=USERNAME,
                    session=HOSTNAME,
                    time=time()-START_TASK_TIME,
                    item_id=task_result.item_id,
                    label=task_result.label,
                    task=task_result.task,
                    annotation_mode=task_result.annotation_mode,
                    viewing_info=task_result.viewing_info,
                    commit_info=COMMIT_ID[0],
                    comments=task_result.comment
                    )

In [ ]:
title_box = ipw.HTML(value='<h1> Welcome {}</h1>'.format(USERNAME.replace('_', ' ')))
if BINARY_TASK:
    task_widget = BinaryClassTask(category_ids, 
                                  annotation_task,
                                  unknown_option=UNKNOWN_OPTION,
                                  max_count=ITEMS_TO_PROCESS)
else:
    task_widget = MultiClassTask(category_ids, annotation_task,
                                 max_count=ITEMS_TO_PROCESS)
task_widget.on_submit(submit_label)
ipw.VBox([title_box,
          task_widget.get_widget()])